In [1]:

import os
os.environ['JAX_PLATFORMS'] = "cpu"  # use "cuda" if you have a GPU
os.environ['JAX_ENABLE_X64'] = "1"


import jax.numpy as jnp
from jnlr.utils.samplers import langevin_implicit
from jnlr.utils.plot_utils import plot_3d_projection

# Parameters as before
R         = 1.0
stroke    = 0.08
epsilon   = 0.06
extrude_y = 0.20
extrude_z = 0.20
z_center_R = -0.40
high = 0.5
width = 0.4
offset=0.2
# Updated segments for N, L and rho (same as before)
N_segments = [
    (jnp.array([-width, -high]), jnp.array([-width,  high])),
    (jnp.array([ width+offset,  high]), jnp.array([ width+offset, -high])),
    (jnp.array([-width,  high]), jnp.array([ width+offset, -high]))
]
L_segments = [
    (jnp.array([width+offset, -high]), jnp.array([width+offset,  high])),
    (jnp.array([-width, -high]), jnp.array([width+offset, -high]))
]
R_segments = [
    (jnp.array([-width, -high]), jnp.array([-width,  high])),
    (jnp.array([-width,  high]), jnp.array([ 0.0,  high])),
    (jnp.array([ 0.0,  high]), jnp.array([ 0.0,  0.2]))
]

def sd_segment(p, a, b):
    pa = p - a
    ba = b - a
    ba_dot = jnp.dot(ba, ba)
    t_clamp = jnp.clip(jnp.dot(pa, ba) / (ba_dot + 1e-8), 0.0, 1.0)
    proj = a + t_clamp * ba
    return jnp.linalg.norm(p - proj, axis=-1)

def smooth_indicator(p, segments):
    d = 1e6
    for a, b in segments:
        d = jnp.minimum(d, sd_segment(p, a, b))
    d0 = d - stroke
    return jnp.clip(d0 / epsilon, 0.0, 1e6)



def gate(t, width):
    return jnp.clip(1.0 - jnp.abs(t) / width, 0.0, 1.0)

def s_N(y, z, x):
    uv = jnp.stack([y, z], axis=-1)
    return smooth_indicator(uv, N_segments)  + jnp.maximum(-x, 0)

def s_L(x, z, y):
    uv = jnp.stack([x, z], axis=-1)
    return smooth_indicator(uv, L_segments) + jnp.maximum(-y, 0)

def s_R(x, y, z):
    uv = jnp.stack([x, y], axis=-1)
    return smooth_indicator(uv, R_segments) * gate(z - z_center_R, extrude_z)

softmax = lambda x, y: jnp.log(jnp.exp(50*x)+jnp.exp(50*y))/50

def phi(xyz):
    x, y, z = xyz
    r = x*x + y*y + z*z
    smax = jnp.maximum(
        s_N(y, z, x),
        jnp.maximum(s_L(x, z, y), s_R(x, y, z))
    )
    #sphere_shell = jnp.maximum(R-0.2-r , jnp.maximum(0, r-R))**2
    #sphere_shell = softmax(jnp.array([R-0.2-r, jnp.maximum(0, r-R)]))**2
    sphere_shell = softmax(R-0.02-r , softmax(0, r-R))**2
    #return r - (R - t * s_L(x, z))
    return sphere_shell + s_N(y, z, x) * s_L(x, z, y) + (s_N(y, z, x) * s_L(x, z, y) == 0)
Y_lang = langevin_implicit(phi, n_samples=10000, burn=100, thin=1, sigma=0.1, lam=0, kappa=0.3, R=3.0, tol=0.03)
#fig = plot_3d_projection(Y_lang)
#fig.update_layout(scene_camera=dict(eye=dict(x=0., y=0., z=2.5)))

from jnlr.reconcile import make_solver_alm_optax as make_solver
X = Y_lang
def f_implicit_sphere(v):
    return v[0]**2 + v[1]**2 + v[2]**2 -1

plot_3d_projection(X, f_implicit=f_implicit_sphere, round_cutoff=None, solver_builder=make_solver, n_iterations=3, remove_axes=True, lo=[-1.2], hi=[1.2], orig_size=7, proj_size=7)

In [2]:
import numpy as np

def make_3d_NLR(T=100, thickness=0.02, cube_size=1.0,
                offset=(0.0, 0.0, 0.0), seed=None):
    """
    Generate a point cloud with:
      • N on plane x = ox,
      • L on plane y = oy,
      • R on the top face (z = oz + cube_size), shaped like a real “R”:

          * vertical leg on the left edge of the top face,
          * a diagonal leg ending at the front‑right corner (upper vertex of L),
          * a quarter‑circle loop that closes the loop.

    Parameters
    ----------
    T : int
        Total number of points per shape (N, L, R).  The shape will be split
        roughly equally among the three parts.
    thickness : float
        Isotropic noise amplitude applied to all points in all directions.
    cube_size : float
        Size of the unit cube.
    offset : tuple
        (ox, oy, oz) – the cube's lower‑left‑front corner.
    seed : int or None
        Random seed for reproducibility.

    Returns
    -------
    np.ndarray
        (3*T, 3) array with all points from N, L and R.
    """
    if seed is not None:
        np.random.seed(seed)

    ox, oy, oz = offset
    cs = cube_size                      # shorthand
    top_z = oz + cs                      # z of the top face

    # ------------------------------------------------------------------
    # 1. N – on plane x = ox
    # ------------------------------------------------------------------
    n_vertical = T // 3
    n_diag     = T - 2 * n_vertical

    N_u = np.empty(T)
    N_v = np.empty(T)

    # left vertical (y‑axis)
    N_u[:n_vertical] = 0.0
    N_v[:n_vertical] = np.random.uniform(0, 1, n_vertical)

    # right vertical (y‑axis)
    N_u[n_vertical:2*n_vertical] = 1.0
    N_v[n_vertical:2*n_vertical] = np.random.uniform(0, 1, n_vertical)

    # diagonal (y = z)
    alpha = np.random.uniform(0, 1, n_diag)
    N_u[2*n_vertical:] = alpha
    N_v[2*n_vertical:] = alpha

    N_pts = np.column_stack([
        np.full(T, ox),      # x = ox (exact)
        oy + N_u * cs,       # y
        oz + N_v * cs        # z
    ])

    # ------------------------------------------------------------------
    # 2. L – on plane y = oy
    # ------------------------------------------------------------------
    l_vertical   = T // 2
    l_horizontal = T - l_vertical

    L_u = np.empty(T)
    L_v = np.empty(T)

    # vertical (x‑axis)
    L_u[:l_vertical] = 0.0
    L_v[:l_vertical] = np.random.uniform(0, 1, l_vertical)

    # horizontal (x‑axis)
    L_u[l_vertical:] = np.random.uniform(0, 1, l_horizontal)
    L_v[l_vertical:] = 0.0

    L_pts = np.column_stack([
        ox + L_u * cs,       # x
        np.full(T, oy),      # y = oy (exact)
        oz + L_v * cs        # z
    ])

    # ------------------------------------------------------------------
    # 3. R – on the bottom face (z = oz)
    #        • stem at y = oy (same as L's horizontal)
    #        • two perpendicular segments going from stem into +y
    #        • arc connecting the tops of the perpendiculars
    #        • diagonal from the arc/perpendicular junction to (ox, oy+cs)
    # ------------------------------------------------------------------
    n_stem  = T // 5          # horizontal stem at y = oy
    n_perp1 = T // 10         # right perpendicular segment
    n_perp2 = T // 10         # left perpendicular segment
    n_arc   = T // 4          # semicircular arc (the "bowl")
    n_leg   = T - n_stem - n_perp1 - n_perp2 - n_arc  # diagonal leg

    bottom_z = oz
    h = cs / 4                # height offset for perpendiculars

    # 3a. Horizontal stem at y = oy (coincides with L's horizontal)
    x_stem = np.linspace(ox, ox + cs, n_stem)
    y_stem = np.full_like(x_stem, oy)
    z_stem = np.full_like(x_stem, bottom_z)

    # 3b. Right perpendicular: from (ox + cs, oy) up to (ox + cs, oy + h)
    y_perp1 = np.linspace(oy, oy + h, n_perp1)
    x_perp1 = np.full_like(y_perp1, ox + cs)
    z_perp1 = np.full_like(y_perp1, bottom_z)

    # 3c. Left perpendicular: from (ox + cs/2, oy) up to (ox + cs/2, oy + h)
    y_perp2 = np.linspace(oy, oy + h, n_perp2)
    x_perp2 = np.full_like(y_perp2, ox + cs / 2)
    z_perp2 = np.full_like(y_perp2, bottom_z)

    # 3d. Arc from (ox + cs, oy + h) curving into +y to (ox + cs/2, oy + h)
    #     Semicircle with center at (ox + 3*cs/4, oy + h), radius = cs/4
    theta = np.linspace(0, np.pi, n_arc)   # 0° → 180°
    r     = cs / 4.0
    cx, cy = ox + 3 * cs / 4, oy + h
    x_arc = cx + r * np.cos(theta)    # ox+cs → ox+cs/2
    y_arc = cy + r * np.sin(theta)    # curves up from oy+h through oy+h+r back to oy+h
    z_arc = np.full_like(x_arc, bottom_z)

    # 3e. Diagonal leg from (ox + cs/2, oy + h) to (ox, oy + cs)
    #     Starts where arc meets left perpendicular, ends at N's diagonal endpoint
    t_leg  = np.linspace(0, 1, n_leg)
    x_leg  = ox + cs / 2 - t_leg * cs / 2       # ox + cs/2 → ox
    y_leg  = oy + h + t_leg * (cs - h)          # oy + h → oy + cs
    z_leg  = np.full_like(x_leg, bottom_z)

    R_pts = np.column_stack([
        np.concatenate([x_stem, x_perp1, x_perp2, x_arc, x_leg]),
        np.concatenate([y_stem, y_perp1, y_perp2, y_arc, y_leg]),
        np.concatenate([z_stem, z_perp1, z_perp2, z_arc, z_leg])
    ])

    # ------------------------------------------------------------------
    # 4. Combine all point clouds and add isotropic noise
    # ------------------------------------------------------------------
    all_pts = np.vstack([N_pts, L_pts, R_pts])
    
    # Add isotropic random noise to all points
    noise = np.random.uniform(-thickness, thickness, all_pts.shape)
    all_pts += noise
    
    return all_pts


X = make_3d_NLR(T=1000, thickness=0.05, cube_size=1.5, seed=42,  offset=(-1.2, -0.8, -1.2))

# Shared point among N, L, R is at (ox, oy, oz)
shared_point = (-1.2, -1, -1)

fig = plot_3d_projection(X, f_implicit=f_implicit_sphere, round_cutoff=None, solver_builder=make_solver, n_iterations=2, remove_axes=True, lo=[-1.2], hi=[1.2], orig_size=2, proj_size=1, proj_alpha=0.05, orig_color="purple", shrink_projection=True, n_grid=100)



# Center camera view on the shared point (normalized coordinates)
# The shared point is at the "lower-left-front" corner of the cube
fig.update_layout(
    scene_camera=dict(
        center=dict(x=-0.1, y=-0.1, z=0),  # Shift view toward the shared corner
        eye=dict(x=-1.2, y=-1.2, z=-0.8)  # Camera closer for tighter framing
    ),
    scene=dict(
        domain=dict(x=[0, 1], y=[0, 1]),  # Scene fills entire figure
        aspectmode='cube'
    ),
    title=None,
    showlegend=False,
    margin=dict(l=0, r=0, t=0.1, b=0, pad=0),  # Remove all margins and padding
    width=400,
    height=350,
    paper_bgcolor='rgba(0,0,0,0)',  # Transparent background
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

# Save as PDF (requires kaleido: uv pip install kaleido)
# SVG is true vector format
fig.write_image("logo.svg", width=400, height=350)

# Also save rasterized PNG for quick preview
fig.write_image("logo.png", width=400, height=350, scale=2)
